<a href="https://colab.research.google.com/github/crystalloide/RAG/blob/main/LAB03_FineTuning_vs_Adapters_vs_RAG_Exemple_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB03 : Fine-Tuning vs Adapters vs RAG – Comparaison des 3 approches - OpenAI

## Objectif
- Comparer comment le fine-tuning, les adapters et RAG résolvent le même problème : faire répondre un LLM à des questions spécifiques à un domaine.

## Durée estimée
- ~15 minutes

## Livrables
- Notebook montrant les résultats Q&A des trois stratégies

## Étape 1 : Setup (5 min)

Installation des bibliothèques nécessaires :

In [ ]:
# !pip install -q --upgrade pip

# Installer les packages principaux
!pip install -q openai
!pip install -q datasets
!pip install -q transformers
!pip install -q faiss-cpu
!pip install -q langchain
!pip install -q langchain langchain-community langchain-openai
!pip install -q langchain-huggingface

# Ignorer les erreurs


In [ ]:
!pip install -q -U langchain-chroma

# Ignorer les erreurs

## Étape 2 : création d'une Base de connaissances basique (5 min)

Nous créons une base de connaissances fictive sur l'IA Agentique.

In [ ]:
# Base de connaissances sur l'IA Agentique
kb = [
    "Agentic AI agents use memory, tools, and goals to act.",
    "LangChain and CrewAI are popular frameworks for building AI agents.",
    "Retrieval-Augmented Generation (RAG) improves accuracy by fetching external knowledge.",
    "Agents can iterate and refine their actions based on feedback.",
    "Tool calling allows agents to interact with APIs and external systems.",
    "Les agents d'IA agentiques utilisent la mémoire, des outils et des objectifs pour agir.",
    "LangChain et CrewAI sont des frameworks populaires pour la création d'agents d'IA.",
    "La génération augmentée par récupération (RAG) améliore la précision en récupérant des connaissances externes.",
    "Les agents peuvent itérer et affiner leurs actions en fonction des retours d'information.",
    "L'appel d'outils permet aux agents d'interagir avec des API et des systèmes externes."
]

# Questions de test
questions = [
    "What are the key components of Agentic AI?",
    "Name one framework for AI agents.",
    "How does RAG improve answers?",
    "Quels sont les composants clés de l'IA agentique ?",
    "Citez un framework pour agents d'IA",
    "Comment la RAG améliore-t-elle les réponses ?",
]

print("Knowledge Base (KB):")
for i, doc in enumerate(kb, 1):
    print(f"  {i}. {doc}")

print("\nTest Questions:")
for i, q in enumerate(questions, 1):
    print(f"  {i}. {q}")

## Étape 3 : Fine-Tuning (démo conceptuelle : 5 min)

**Fine-tuning** = consiste à mettre à jour les poids du modèle avec de nouveaux exemples étiquetés.

Voici à quoi ressemblerait un dataset de fine-tuning :

In [ ]:
from datasets import Dataset

# Données d'entraînement pour fine-tuning
train_data = Dataset.from_dict({
    "prompt": [
        "Q: What are the key components of Agentic AI?\nA:",
        "Q: Name one framework for AI agents.\nA:",
        "Q: How does RAG improve answers?\nA:"
    ],
    "completion": [
        " Agentic AI agents use memory, tools, and goals to act.",
        " LangChain is a framework for building AI agents.",
        " RAG improves accuracy by fetching external knowledge before answering."
    ]
})

print("Fine-Tuning Training Dataset:")
print(train_data)
print("\n" + "="*80)
print("FINE-TUNING OBSERVATIONS:")
print("="*80)
print("""
✓ Avantages:
  - Modèle hautement adapté au domaine
  - Connaissances "baked-in" et rapides

✗ Inconvénients:
  - Coûteux (calcul + infrastructure)
  - Rigide (difficile de mettre à jour les connaissances)
  - Nécessite un réentraînement pour chaque nouveau domaine
  - Dépend de la qualité des données d'entraînement

📌 Avec OpenAI ou Hugging Face, vous uploaderiez ce dataset pour fine-tuning.
""")

## Étape 4 : Adapters / LoRA (démo conceptuelle : 5 min)

**Adapters** = petites couches parameter-efficient que l'on entraîne au lieu de ré-entraîner tout le modèle entier.

C'est une alternative au fine-tuning complet.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Charger un petit modèle pour la démo
model_name = "distilgpt2"
try:
    tok = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    total_params = sum(p.numel() for p in model.parameters())
    print(f"✓ Modèle chargé: {model_name}")
    print(f"  Nombre total de paramètres: {total_params:,}")

except Exception as e:
    print(f"Note: Téléchargement du modèle échoué (attendu en mode hors-ligne)")
    print(f"Erreur: {e}")

print("\n" + "="*80)
print("ADAPTERS / LoRA OBSERVATIONS:")
print("="*80)
print("""
✓ Avantages:
  - TRÈS peu de paramètres à entraîner (~0.1-1% du modèle original)
  - Économique (moins de calcul, moins de mémoire)
  - Modulaire (plusieurs adapters pour différents domaines)
  - Rapide à mettre à jour

✗ Inconvénients:
  - Nécessite toujours une infrastructure d'entraînement
  - Performance légèrement inférieure au fine-tuning complet
  - Nécessite un framework compatible (PEFT, AdapterHub)

📌 Avec LoRA, vous ne mettriez à jour que quelques millions de paramètres
    au lieu de milliards, rendant l'entraînement beaucoup plus efficace.
""")

## Étape 5 : RAG – démo intéractive (5 min)

Contrairement à l'entraînement, **RAG** récupère les connaissances externes au moment de l'exécution.

Construisons un système RAG complet.

### Configuration de RAG avec LangChain et ChromaDB

Nous utilisons ChromaDB (une alternative légère à FAISS) pour cette démo :

In [ ]:
# from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import os

print("LangChain et ChromaDB importés avec succès.")
print("\n📌 Configuration du système RAG...")

### Construction d'un RAG avec OpenAI (nécessite une clé API valide OPENAI_API_KEY)

In [ ]:
import os
from google.colab import userdata

# Get API key from Colab Secrets (add it in the Secrets manager: 🔑 icon on left panel)
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# Or if running locally with .env file:
# from dotenv import load_dotenv
# load_dotenv()

print("✓ API Key configured successfully")


### Démonstration du fonctionnement du RAG : Retrieval (Récupération de documents)

In [ ]:
def setup_rag_with_openai():
    """
    Configure RAG avec OpenAI embeddings et ChatOpenAI - VERSION MODERNE.
    Nécessite une clé API OpenAI.
    """
    import os
    from getpass import getpass

    # Vérifier ou demander la clé API
    if not os.getenv("OPENAI_API_KEY"):
        print("\n🔑 Clé API OpenAI requise pour cette démo.")
        print("   Créez une clé sur https://platform.openai.com/api-keys")
        api_key = getpass("Entrez votre OPENAI_API_KEY: ")
        os.environ["OPENAI_API_KEY"] = api_key

    from langchain_community.vectorstores import Chroma
    from langchain_openai import OpenAIEmbeddings, ChatOpenAI
    from langchain_core.documents import Document
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.runnables import RunnablePassthrough
    from langchain_core.output_parsers import StrOutputParser

    try:
    # 🗑️ Supprimer la base Chroma existante
    #    chroma_dir = "/content/chroma_db"
    #    if os.path.exists(chroma_dir):
    #        print(f"🗑️  Suppression de la base existante: {chroma_dir}")
    #        shutil.rmtree(chroma_dir)
    #        print("✓ Base supprimée")
        print("📚 Création des documents...")
        docs = [Document(page_content=x) for x in kb]

        print("🔨 Initialisation des embeddings OpenAI...")
        embeddings = OpenAIEmbeddings()

        print("💾 Construction de la base vectorielle...")
        db = Chroma.from_documents(docs, embeddings)
        #db = Chroma.from_documents(docs, embeddings, persist_directory=chroma_dir)
        retriever = db.as_retriever(search_kwargs={"k": 2})

        print("🤖 Configuration de la chaîne RAG...")

        # Template de prompt
        template = """Réponds à la question en te basant sur le contexte suivant :

Contexte: {context}

Question: {question}

Réponse:"""

        prompt = ChatPromptTemplate.from_template(template)
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

        # Fonction pour formater les documents
        def format_docs(docs):
            return "\n\n".join([doc.page_content for doc in docs])

        # Chaîne RAG moderne avec LCEL
        qa_chain = (
            {
                "context": retriever | format_docs,
                "question": RunnablePassthrough()
            }
            | prompt
            | llm
            | StrOutputParser()
        )

        print("✅ Système RAG configuré avec succès!\n")
        return qa_chain, db

    except Exception as e:
        print(f"⚠️ Erreur lors de la configuration RAG: {e}")
        return None, None

print("✓ Fonction setup_rag_with_openai() définie (version moderne LCEL).")

### Full RAG Pipeline avec Q&A Complète

Maintenant, montrons une pipeline Q&A RAG complète :

In [ ]:
# Utilisation d'OpenAI
import os
if os.getenv("OPENAI_API_KEY"):
    qa, db = setup_rag_with_openai()

    print("\n" + "="*80)
    print("FULL RAG Q&A avec ChatOpenAI (gpt-4o-mini)")
    print("="*80)

    for question in questions:
        print(f"\n❓ Q: {question}")
        answer = qa.invoke(question)
        print(f"✅ A: {answer}")

    # Question bonus
    bonus_q = "Explain Agentic AI like I'm 10 years old."
    print(f"\n❓ BONUS Q: {bonus_q}")
    bonus_answer = qa.invoke(bonus_q)
    print(f"✅ A: {bonus_answer}")
else:
    print("💡 Pour la démo :")
    print("   1. Créez une clé sur https://platform.openai.com/api-keys")
    print("   2. Entrez votre clé valide API")
##"""

print("\n✓ Code RAG avec OpenAI .")

## Étape 6 : Comparaison des 3 méthodes (5 min)

Créons un tableau comparatif des trois approches :

In [ ]:
import pandas as pd
from IPython.display import display, HTML

# Créer le tableau comparatif
comparison_data = {
    "Aspect": [
        "Accuracy",
        "Cost",
        "Training Time",
        "Update Frequency",
        "Knowledge Freshness",
        "Scalability",
        "Complexity",
        "Infrastructure"
    ],
    "Fine-Tuning": [
        "⭐⭐⭐⭐⭐",
        "💰💰💰💰",
        "🕐 Hours/Days",
        "Monthly/Quarterly",
        "Stale (until retrained)",
        "Fair (one model per domain)",
        "🔴 High",
        "GPUs required"
    ],
    "Adapters/LoRA": [
        "⭐⭐⭐⭐",
        "💰💰",
        "🕐 Minutes/Hours",
        "Weekly",
        "Semi-fresh",
        "Good (modular)",
        "🟡 Medium",
        "GPUs (lighter)"
    ],
    "RAG": [
        "⭐⭐⭐",
        "💰",
        "⚡ Real-time",
        "Real-time",
        "✨ Always Fresh",
        "Excellent",
        "🟢 Low",
        "Vector DB + API"
    ]
}

df = pd.DataFrame(comparison_data)

# Création du HTML stylisé
html = """
<style>
    .comparison-table {
        width: 100%;
        max-width: 1200px;
        margin: 20px auto;
        background: linear-gradient(to bottom right, #f8fafc, #eff6ff);
        border-radius: 8px;
        overflow: hidden;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    }
    .table-header {
        background: linear-gradient(to right, #2563eb, #4f46e5);
        padding: 20px;
        text-align: center;
    }
    .table-header h1 {
        color: white;
        font-size: 24px;
        font-weight: bold;
        margin: 0;
    }
    .styled-table {
        width: 100%;
        border-collapse: collapse;
        background: white;
    }
    .styled-table thead tr {
        background-color: #f1f5f9;
        border-bottom: 2px solid #cbd5e1;
    }
    .styled-table th {
        padding: 16px;
        text-align: left;
        font-weight: 600;
        color: #334155;
        width: 25%;
    }
    .styled-table td {
        padding: 16px;
        color: #475569;
        border-bottom: 1px solid #e2e8f0;
    }
    .styled-table tbody tr:nth-child(even) {
        background-color: #f8fafc;
    }
    .styled-table tbody tr:hover {
        background-color: #dbeafe;
        transition: background-color 0.2s;
    }
    .styled-table tbody tr td:first-child {
        font-weight: 500;
        color: #1e293b;
    }
    .table-footer {
        background-color: #f8fafc;
        padding: 20px;
        border-top: 1px solid #e2e8f0;
    }
    .footer-grid {
        display: grid;
        grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
        gap: 16px;
        font-size: 14px;
    }
    .footer-item {
        display: flex;
        gap: 8px;
        color: #1e40af;  /* Bleu foncé pour tout le texte */
    }
    .footer-item strong {
        color: #1e3a8a;  /* Bleu foncé */
    }
    .footer-item span:first-child {
        font-size: 18px;
    }
</style>

<div class="comparison-table">
    <div class="table-header">
        <h1>Tableau Comparatif: Fine-Tuning vs Adapters/LoRA vs RAG</h1>
    </div>

    <table class="styled-table">
        <thead>
            <tr>
                <th>Aspect</th>
                <th>Fine-Tuning</th>
                <th>Adapters/LoRA</th>
                <th>RAG</th>
            </tr>
        </thead>
        <tbody>
"""

# Ajouter les lignes du tableau
for _, row in df.iterrows():
    html += f"""
            <tr>
                <td>{row['Aspect']}</td>
                <td>{row['Fine-Tuning']}</td>
                <td>{row['Adapters/LoRA']}</td>
                <td>{row['RAG']}</td>
            </tr>
"""

html += """
        </tbody>
    </table>

    <div class="table-footer">
        <div class="footer-grid">
            <div class="footer-item">
                <span>💡</span>
                <div><strong>Fine-Tuning:</strong> Meilleure précision mais coûteux</div>
            </div>
            <div class="footer-item">
                <span>💡</span>
                <div><strong>Adapters/LoRA:</strong> Bon compromis coût/performance</div>
            </div>
            <div class="footer-item">
                <span>💡</span>
                <div><strong>RAG:</strong> Données fraîches, faible complexité</div>
            </div>
        </div>
    </div>
</div>
"""

# Afficher le tableau stylisé
display(HTML(html))

### Tableau Détaillé des Cas d'Usage

In [ ]:
from IPython.display import HTML, display

# Create the DataFrame
df_usecases = pd.DataFrame({
    'Method': [
        'Fine-Tuning',
        'Adapters (LoRA)',
        'RAG'
    ],
    'Pros': [
        '✓ Highly accurate\n✓ Baked-in knowledge\n✓ Very fast inference',
        '✓ Cheap fine-tuning\n✓ Modular/multi-domain\n✓ Fast training',
        '✓ Flexible\n✓ Real-time updates\n✓ No retraining'
    ],
    'Cons': [
        '✗ Expensive\n✗ Rigid (hard to update)\n✗ Retrain per domain',
        '✗ Still needs training\n✗ Slightly lower accuracy\n✗ Framework dependent',
        '✗ Retriever quality matters\n✗ Latency (retrieval step)\n✗ Context window limits'
    ],
    'Best Use Case': [
        'Applications à domaine restreint (par exemple : chatbot médical)',
        'Adaptation au domaine (tâches spécialisées multiples)',
        'Connaissances dynamiques (actualités, FAQ, données en temps réel)'
    ]
})


html_table = """
<table style="width:100%; border-collapse: collapse; font-family: monospace;">
<tr style="background-color: #aaaaaa;">
    <th style="border: 1px solid #ddd; padding: 16px; color: Black; text-align: left;">Méthode</th>
    <th style="border: 1px solid #ddd; padding: 16px; color: black; text-align: left;">Avantages</th>
    <th style="border: 1px solid #ddd; padding: 16px; color: black; text-align: left;">Inconvénients</th>
    <th style="border: 1px solid #ddd; padding: 16px; color: black; text-align: left;">Cas d'usage</th>
</tr>
"""

for idx, row in df_usecases.iterrows():
    pros_html = "<br>".join(row['Pros'].split('\n'))
    cons_html = "<br>".join(row['Cons'].split('\n'))
    use_html = "<br>".join(row['Best Use Case'].split('\n'))

    html_table += f"""
<tr>
    <td style="border: 1px solid #ddd; padding: 10px; color: white; font-weight: bold;">{row['Method']}</td>
    <td style="border: 1px solid #ddd; padding: 10px; color: green;">{pros_html}</td>
    <td style="border: 1px solid #ddd; padding: 10px; color: red;">{cons_html}</td>
    <td style="border: 1px solid #ddd; padding: 10px;">{use_html}</td>
</tr>
"""

html_table += "</table>"
display(HTML(html_table))


## Résumé et Conclusions

In [ ]:
print("""
╔═══════════════════════════════════════════════════════════════════════════════╗
║              LAB03 - CONCLUSIONS & KEY TAKEAWAYS                              ║
╚═══════════════════════════════════════════════════════════════════════════════╝

🎯 TROIS STRATÉGIES POUR ADAPTER LES LLMs À UN DOMAINE:

1️⃣  FINE-TUNING
   └─ Entraîner le modèle sur de nouvelles données
   └─ ✅ Très précis | ❌ Coûteux, rigide
   └─ 💡 Meilleur pour: Applications de niche

2️⃣  ADAPTERS / LoRA
   └─ Entraîner seulement de petits modules paramétriques
   └─ ✅ Économique, modulaire | ❌ Nécessite toujours du training
   └─ 💡 Meilleur pour: Adaptation multi-domaine

3️⃣  RAG (Retrieval-Augmented Generation)
   └─ Récupérer des connaissances externes en temps réel
   └─ ✅ Flexible, mises à jour faciles | ❌ Dépend de la qualité du retriever
   └─ 💡 Meilleur pour: Connaissances dynamiques, agents IA

╔═══════════════════════════════════════════════════════════════════════════════╗
║                    🚀 RECOMMANDATION POUR LES AGENTS IA                       ║
╚═══════════════════════════════════════════════════════════════════════════════╝

Pour les Agentic AI systems (LangChain, CrewAI, etc.):

   ➡️  RAG est souvent le GO-TO car:
      • Les agents ont besoin de connaissances à jour
      • Les tools et l'itération demandent de la flexibilité
      • Les coûts sont raisonnables vs fine-tuning

   ➡️  Combinaison optimale:
      • RAG pour les connaissances métier dynamiques
      • Fine-tuning/Adapters pour le format/style du modèle
      • Tool calling pour les actions externes

╔═══════════════════════════════════════════════════════════════════════════════╗
║                         ✅ NEXT STEPS                                         ║
╚═══════════════════════════════════════════════════════════════════════════════╝

1. Pratiquez RAG avec votre propre base de connaissances
2. Explorez les frameworks (LangChain, Llama Index, etc.)
3. Optimisez le retriever avec différentes embeddings
4. Testez les agents IA intégrant ces stratégies
5. Mesurez la qualité des réponses (eval)

""")

print("\n" + "="*80)
print("🎉 LAB03 Finalisé !")
print("="*80)

## Resources Supplémentaires

In [ ]:
resources = """
📚 RESSOURCES RECOMMANDÉES:

RAG & LangChain:
  • https://python.langchain.com/
  • https://js.langchain.com/
  • RAG Best Practices: https://docs.llamaindex.ai/

Fine-Tuning & LoRA:
  • HuggingFace Fine-Tuning: https://huggingface.co/docs/transformers/training
  • PEFT (LoRA): https://github.com/huggingface/peft
  • OpenAI Fine-Tuning: https://platform.openai.com/docs/guides/fine-tuning

Vector Databases:
  • Chroma: https://www.trychroma.com/
  • Pinecone: https://www.pinecone.io/
  • Weaviate: https://weaviate.io/

Frameworks Agentic AI:
  • LangChain: https://python.langchain.com/
  • CrewAI: https://www.crewai.com/
  • Llama Index: https://www.llamaindex.ai/

Évaluation des systèmes RAG:
  • RAGAS: https://github.com/explodinggradients/ragas
  • DeepEval: https://www.deepeval.com/
"""
print(resources)